In [36]:
import datetime
import pandas as pd
import numpy as np
import yfinance as yf


# for plotting purposes 
from datetime import datetime
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
# Equivalent timeframe mappings
frame_M1 = '1m'
frame_M15 = '15m'
frame_M30 = '30m'
frame_H1 = '1h'
frame_H4 = '4h'
frame_D1 = '1d'
frame_W1 = '1wk'


# Matching your MT5 asset list with Yahoo tickers
assets = {
    'EURUSD': 'EURUSD=X',
    'USDCHF': 'USDCHF=X',
    'GBPUSD': 'GBPUSD=X',
    'USDCAD': 'USDCAD=X',
    'BTCUSD': 'BTC-USD',
    'ETHUSD': 'ETH-USD',
    'XAUUSD': 'XAUUSD=X',
    'XAGUSD': 'XAGUSD=X',
    'SP500m': '^GSPC',
    'UK100': '^FTSE'
}

In [38]:
def get_quotes(time_frame,  period='2yrs', asset='EURUSD'):
    try:
        symbol = assets[asset]
        df = yf.download(
            symbol,
            period=period,
            interval=time_frame,
        )

        if df.empty:
            print(f"No data for {asset}. Check ticker symbol or timeframe.")
            return pd.DataFrame()

        df.reset_index(inplace=True)
        return df

    except Exception as e:
        print(f"Error fetching data for {asset}: {e}")
        return pd.DataFrame()


def mass_import(asset_index, time_frame):

    asset_list = list(assets.keys())
    asset = asset_list[asset_index]

    frame_M15 = '15m'
    frame_M30 = '30m'
    frame_H1 = '1h'
    frame_H4 = '4h'
    frame_D1 = '1d'
    frame_W1 = '1wk'

    if time_frame == '1WK':
        data = get_quotes(frame_W1, '10y', asset=asset)
    elif time_frame == 'D1':
        data = get_quotes(frame_D1, '5y', asset=asset)
    elif time_frame == '4H':
        data = get_quotes(frame_H4, '2y', asset=asset)
    elif time_frame == '1H':
        data = get_quotes(frame_H1, '2y', asset=asset)
    elif time_frame == 'M30':
        data = get_quotes(frame_M30, '60d', asset=asset)
    elif time_frame == 'M15':
        data = get_quotes(frame_M15, '60d', asset=asset)
    else:
        data = get_quotes(frame_D1, 'D1', asset=asset)

    if data.empty:
        print(f"No data found for {asset} on timeframe {time_frame}")
        return np.array([])

    return data


def flatten_yf_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Removes the 'Ticker' row level from a Yahoo Finance DataFrame
    that has multi-index columns like ('EURUSD=X', 'Open').

    Returns a DataFrame with simple column names like 'Open', 'High', etc.
    """
    # If columns are MultiIndex (e.g. ('EURUSD=X', 'Open')), flatten them
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.droplevel(1)
    return df


def detect_fvg(data, lookback_period=14, body_multiplier=1.5):
    """
    Detects Fair Value Gaps (FVGs) in historical price data.

    Rules Added:
    - Middle candle body must be > avg body * body_multiplier
    - The FVG gap must be > 10% of the middle candle body
    """

    fvg_list = [None, None]

    for i in range(2, len(data)):
        first_high = data['High'].iloc[i-2]
        first_low = data['Low'].iloc[i-2]
        middle_open = data['Open'].iloc[i-1]
        middle_close = data['Close'].iloc[i-1]
        third_low = data['Low'].iloc[i]
        third_high = data['High'].iloc[i]

        # Middle candle body
        middle_body = abs(middle_close - middle_open)

        # Avg body over lookback
        prev_bodies = (data['Close'].iloc[max(0, i-1-lookback_period):i-1] -
                       data['Open'].iloc[max(0, i-1-lookback_period):i-1]).abs()
        avg_body_size = prev_bodies.mean()
        avg_body_size = avg_body_size if avg_body_size > 0 else 0.001

        # Minimum required gap size (10% of middle body)
        min_gap = middle_body * 0.10

        # -----------------------------
        # Check Bullish FVG
        # -----------------------------
        if third_low > first_high:
            gap_size = third_low - first_high

            if (middle_body > avg_body_size * body_multiplier
                    and gap_size > min_gap):

                fvg_list.append(('bullish', first_high, third_low, i))
                print("bullish", first_high, third_low, i)
                continue

        # -----------------------------
        # Check Bearish FVG
        # -----------------------------
        if third_high < first_low:
            gap_size = first_low - third_high

            if (middle_body > avg_body_size * body_multiplier
                    and gap_size > min_gap):

                fvg_list.append(('bearish', first_low, third_high, i))
                print("bearish", first_low, third_high, i)
                continue

        # No FVG
        fvg_list.append(None)

    return fvg_list

In [39]:
dataF = mass_import(0, '4H')
dataF = dataF.reset_index()            # Move Datetime from index to column
dataF.rename(columns={'Date': 'Gmt time'}, inplace=True)


try:
    my_data = dataF[['Datetime', 'Open', 'High', 'Low', 'Close', 'Volume']]
except:
    my_data = dataF[['Gmt time', 'Open', 'High', 'Low', 'Close', 'Volume']]
    
    
my_data = flatten_yf_columns(my_data)


my_data['FVG'] = detect_fvg(my_data)


/var/folders/pp/x__1tf4n7j98nw_8bwlmgxj00000gn/T/ipykernel_8371/1484531969.py:4: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed

bullish 1.09505033493042 1.0954102277755737 7
bearish 1.0934937000274658 1.0921800136566162 11
bullish 1.0922993421554565 1.093135118484497 28
bullish 1.0960105657577515 1.0976948738098145 40
bearish 1.0970927476882935 1.0961307287216187 50
bearish 1.0860122442245483 1.0831888914108276 64
bearish 1.07758629322052 1.0772379636764526 88
bullish 1.0810810327529907 1.08695650100708 107
bullish 1.0933741331100464 1.0982975959777832 112
bearish 1.095170259475708 1.091941475868225 118
bullish 1.09505033493042 1.0973334312438965 130
bullish 1.0958904027938843 1.0978153944015503 141
bullish 1.1035091876983643 1.1042402982711792 159
bullish 1.1061947345733643 1.1102476119995117 164
bearish 1.1019283533096313 1.0974539518356323 183
bearish 1.0969723463058472 1.0963709354400635 184
bullish 1.0952903032302856 1.0955302715301514 187
bearish 1.0955302715301514 1.0939722061157227 189
bearish 1.0942115783691406 1.093135118484497 200
bearish 1.0948106050491333 1.0928961038589478 241
bearish 1.0912265777

In [40]:
my_data.iloc[3125]

Price
Datetime                            2025-11-19 16:00:00+00:00
Open                                                 1.155001
High                                                 1.155001
Low                                                  1.153802
Close                                                1.154335
Volume                                                      0
FVG         (bearish, 1.1570056676864624, 1.15500116348266...
Name: 3125, dtype: object

In [41]:
dfpl = my_data.tail(100)

# Create the figure
fig = go.Figure()

# Add candlestick chart
fig.add_trace(go.Candlestick(
    x=dfpl.index,
    open=dfpl["Open"],
    high=dfpl["High"],
    low=dfpl["Low"],
    close=dfpl["Close"],
    name="Candles"
))

# Add FVG zones
for _, row in dfpl.iterrows():
    if isinstance(row["FVG"], tuple):
        fvg_type, start, end, index = row["FVG"]
        print('FVG LEVEL *****', row["FVG"])
        color = "rgba(0,255,0,0.3)" if fvg_type == "bullish" else "rgba(255,0,0,0.3)"
        fig.add_shape(
            type="rect",
            x0=index - 2,
            x1=index + 30,
            y0=start,
            y1=end,
            fillcolor=color,
            opacity=0.8,
            layer="below",
            line=dict(width=0),
        )

# Show the chart
fig.update_layout(width=1200, height=800,
                  xaxis=dict(showgrid=False),
                  yaxis=dict(showgrid=False),
                  plot_bgcolor='black',
                  paper_bgcolor='black')
fig.show()

FVG LEVEL ***** ('bearish', np.float64(1.1652295589447021), np.float64(1.1640088558197021), 3032)
FVG LEVEL ***** ('bearish', np.float64(1.1631964445114136), np.float64(1.161440134048462), 3036)
FVG LEVEL ***** ('bearish', np.float64(1.161440134048462), np.float64(1.1602274179458618), 3040)
FVG LEVEL ***** ('bearish', np.float64(1.1599582433700562), np.float64(1.1574074029922485), 3041)
FVG LEVEL ***** ('bearish', np.float64(1.156203031539917), np.float64(1.1547343730926514), 3047)
FVG LEVEL ***** ('bearish', np.float64(1.1535356044769287), np.float64(1.1528706550598145), 3052)
FVG LEVEL ***** ('bearish', np.float64(1.151145339012146), np.float64(1.1499539613723755), 3058)
FVG LEVEL ***** ('bullish', np.float64(1.1526048183441162), np.float64(1.1535356044769287), 3071)
FVG LEVEL ***** ('bullish', np.float64(1.1542012691497803), np.float64(1.1558021306991577), 3076)
FVG LEVEL ***** ('bullish', np.float64(1.158077597618103), np.float64(1.1587486267089844), 3089)
FVG LEVEL ***** ('bullish